In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Pauli
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
import warnings
import uuid

warnings.filterwarnings('ignore')

# Initialize the Runtime Service
service = QiskitRuntimeService()

##############################################################################
# 1. Create PARTIALLY entangled state:
#    cos(theta/2)|00> + sin(theta/2)|11>
##############################################################################
def create_entangled_state(theta):
    qc = QuantumCircuit(2)
    qc.ry(theta, 0)  
    qc.cx(0, 1)
    return qc

##############################################################################
# 2. Measure <Z ⊗ Z> after local R_y() rotations on each qubit
##############################################################################
def measure_correlation(counts, angle_qubit_0, angle_qubit_1):
    outcomes = {'00': 0, '01': 0, '10': 0, '11': 0}
    
    # Transform angles into measurement bases
    measurement_mapping = {
        'A1': angle_qubit_0,
        'A2': angle_qubit_0 + np.pi/2,
        'B1': angle_qubit_1,
        'B2': angle_qubit_1 + np.pi/2
    }
    
    for outcome, count in counts.items():
        outcomes[outcome] += count
    
    # Compute the correlation <Z ⊗ Z>
    correlation = (outcomes['00'] + outcomes['11'] - outcomes['01'] - outcomes['10']) / sum(outcomes.values())
    return correlation

##############################################################################
# 3. Compute CHSH S-value
##############################################################################
def chsh_s_value(counts, angles):
    A1B1 = measure_correlation(counts, angles['A1'], angles['B1'])
    A1B2 = measure_correlation(counts, angles['A1'], angles['B2'])
    A2B1 = measure_correlation(counts, angles['A2'], angles['B1'])
    A2B2 = measure_correlation(counts, angles['A2'], angles['B2'])
    return abs(A1B1 + A1B2 + A2B1 - A2B2)

##############################################################################

if __name__ == "__main__":
    thetas = np.linspace(0, 2 * np.pi, 100)
    print(f"Total thetas: {len(thetas)}")

    # Automatically select the least busy backend
    backend = service.least_busy()

    circuits = []

    for idx, theta in enumerate(thetas):
        qc = create_entangled_state(theta)
        qc.measure_all()  # Measure all qubits
        qc.name = f"theta_{idx}_{uuid.uuid4()}"
        circuits.append(qc)

    transpiled_circuits = transpile(circuits, backend=backend)
    
    # Use Sampler to submit the circuits
    sampler = Sampler(backend)
    job = sampler.run(transpiled_circuits, shots=1024)  # Define number of shots
    result = job.result()

    a1_values = np.linspace(0, 2 * np.pi, 15)
    b1_values = np.linspace(0, 2 * np.pi, 15)

    max_s_list = [] 
    all_s_vals = []

    for idx, theta in enumerate(thetas):
        qc_name = transpiled_circuits[idx].name
        counts = result.data(qc_name)['counts']

        best_s = float('-inf')
        best_angles = None

        for A1 in a1_values:
            A2_options = [A1 + np.pi / 2, A1 - np.pi / 2]
            for A2 in A2_options:
                for B1 in b1_values:
                    B2_options = [B1 + np.pi / 2, B1 - np.pi / 2]
                    for B2 in B2_options:
                        angles = {'A1': A1, 'A2': A2, 'B1': B1, 'B2': B2}
                        s_val = chsh_s_value(counts, angles)
                        all_s_vals.append(s_val)
                        if s_val > best_s:
                            best_s = s_val
                            best_angles = angles
        
        max_s_list.append({
            'theta': theta,
            'max_S': best_s,
            'angles': best_angles
        })

        print(f"Theta = {theta:.3f}, Max S = {best_s:.3f}, Best Angles = {best_angles}")

    normalized_thetas = thetas / np.pi

    # Plot Results
    import matplotlib.ticker as tck

    fig, ax = plt.subplots(figsize=(7, 5))

    ax.plot(normalized_thetas, [result['max_S'] for result in max_s_list], "o-", label="CHSH S-value", zorder=3)

    ax.axhline(y=2, color="0.9", linestyle="--", label="Classical Bound (S=2)")
    ax.axhline(y=2 * np.sqrt(2), color="0.9", linestyle="-.", label="Tsirelson Bound (S≈2.828)")
    ax.fill_between(normalized_thetas, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7, label="Quantum Violation Range")

    ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
    ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

    plt.xlabel("Angle for Degree of Entanglement ($\\theta / \\pi$)")
    plt.ylabel("CHSH S-value")
    plt.title("CHSH Violation vs. Entanglement Angle $\\theta$")
    plt.legend()
    plt.grid(True)

    plt.show()


Total thetas: 100
